<a href="https://colab.research.google.com/github/gopalm-ai/qiskit-tutorials/blob/master/HHL_Qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HHL Algorithm

The HHL quantum algorithm was proposed by Harrow, Hassidim, and Lloyd. The goal of the algorithm is to solve a linear system *Ax* = *b* by expressing the vector x as a quantum state |x> and the vector *b* as a quantum state |b> over log_2(N) qubits.

---



---



In [2]:
! pip3 install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 8.4 MB/s 
     |████████████████████████████████| 18.0 MB 351 kB/s 
     |████████████████████████████████| 240 kB 65.4 MB/s 
     |████████████████████████████████| 112 kB 66.8 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 49 kB 6.8 MB/s 
     |████████████████████████████████| 38.1 MB 316 kB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 58.9 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 943 kB 44.3 MB/s 
     |████████████████████████████████| 4.0 MB 41.3 MB/s 
     |████████████████████████████████| 112 kB 63.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.37.0-py3-none-any.whl size=12028 sha256=906b3839fa16c0d72c1e523f1be00cec94cb2ed8b2b4d02abe46aa72352d7d78
  S

In [3]:
import numpy as np
from qiskit import *
pi = np.pi
q = QuantumRegister(6)
c = ClassicalRegister(2)
hhl = QuantumCircuit(q, c)

The algorithm performs the following actions:
1) Extract the eigenvalues of A via quantum phase estimation
2) Controlled rotations of ancilla qubit
3) Uncompute with inverse quantum phase estimation

## Superposition and Quantum Circuit Definition

The input register begins with a superposition of eigenvectors. The initial aim is to create a superposition of A as a Hamiltonian for different time durations.

In [4]:
# Superposition
hhl.h(q[1])
hhl.h(q[2])
# Controlled-U0
hhl.cu3(-pi/2, -pi/2, pi/2, q[2], q[3])
# hhl.cu1(3*pi/4, q[2], q[3])
hhl.cp(3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
# hhl.cu1(3*pi/4, q[2], q[3])
hhl.cp(3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
# Controlled-U1
hhl.cx(q[1], q[3])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The QuantumCircuit.cu3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cu method instead, where cu3(ϴ,φ,λ) = cu(ϴ,φ,λ,0).
  """


## Inverse QFT

Apply the quantum Inverse Fourier Transform to write the phase to a register.

In [5]:
hhl.swap(q[1], q[2])
hhl.h(q[2])
hhl.cp(-pi/2, q[1], q[2])
hhl.h(q[1])

In [6]:
# Ancilla Rotation 

In [7]:
hhl.cu(0.392699, 0, 0, 0, q[1], q[0]) # Controlled-RY0
hhl.cu(0.19634955, 0, 0, 0, q[2], q[0]) # Controlled-RY1

Now perform operations on the phase to invert it. In other words, uncompute all operations (except for those that store the desired information in the final registers).

In [8]:
# QPE Uncompute Action

In [9]:
hhl.swap(q[1], q[2])
hhl.h(q[1])
hhl.cp(pi/2, q[1], q[2]) # Inverse(Dagger(Controlled-S))
hhl.h(q[2])
hhl.swap(q[2], q[1])
# Inverse(Controlled-U1)
hhl.cx(q[1], q[3])
# Inverse(Controlled-U0)
hhl.cx(q[2], q[3])
hhl.cp(-3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
hhl.cp(-3*pi/4, q[2], q[3])
hhl.cu(-pi/2, pi/2, -pi/2, 0, q[2], q[3])
# End of Inverse(controlled-U0)
hhl.h(q[2])
hhl.h(q[1])

Now prepare correct output state to perform swap test with outcome. 

In [10]:
# Post-selection

In [11]:
# Target state preparation
hhl.rz(-pi, q[4])
hhl.p(pi, q[4])
hhl.h(q[4])
hhl.ry(-0.9311623288419387, q[4])
hhl.rz(pi, q[4])
# Swap test
hhl.h(q[5])
hhl.cx(q[4], q[3])
hhl.ccx(q[5], q[3], q[4])
hhl.cx(q[4], q[3])
hhl.h(q[5])
hhl.barrier(q)
hhl.measure(q[0], c[0])
hhl.measure(q[5], c[1])

Measure ancilla register for post-selection, and measure result of swap test. Define helper functions to calculate success probability.

In [12]:
# Success Functions

In [13]:
def get_psuccess(counts):

  try:
    succ_rotation_fail_swap = counts['11']
  except KeyError:
    succ_rotation_fail_swap = 0
  try:
    succ_rotation_succ_swap = counts['01']
  except KeyError:
    succ_rotation_succ_swap = 0
  succ_rotation = succ_rotation_succ_swap + succ_rotation_fail_swap
  try:
    prob_swap_test_success = succ_rotation_succ_swap / succ_rotation
  except ZeroDivisionError:
    prob_swap_test_success = 0
  return prob_swap_test_success

Run circuit on simulator.

In [14]:
# Run circuit

In [15]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(hhl, backend, shots=100)
result = job.result()
counts = result.get_counts(hhl)
print(get_psuccess(counts))

0.0


In [16]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright